In [1]:
#using Revise
using DrWatson
@quickactivate "ABoxWorld"
include(srcdir("ABoxWorld.jl"));

┌ Info: ABoxWorld project environment is loaded and active
└ @ Main s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\src\ABoxWorld.jl:10


In [2]:

using LinearAlgebra


In [3]:

global_eps_tol = 1e-10

dynamic_slice(A, dims) = A[ntuple(d -> (d in dims) ? (1:1) : (:), ndims(A))...] #Slices the first elements from A in any dimension in dims and returns other dimensions in full

mutable struct TimeOrderedNSBox
    bell_scenario::NTuple{4, <:Int} #(M_A, M_B, m_A, m_B)
    seq_scenario::NTuple{2, <:Int} #(s,t)
    condit_joint_dist::Array{Float64}

    function TimeOrderedNSBox(;bell_scenario::NTuple{4, <:Int}, seq_scenario::NTuple{2, <:Int}, condit_joint_dist::Array{<:Real})
        """ Constructor with AoT and NS constraints check"""
        @assert all(length(size(condit_joint_dist)) == 2*seq_scenario[1] + 2*seq_scenario[2]) 
        @assert all(size(condit_joint_dist) == (Tuple(bell_scenario[3] for _ in 1:seq_scenario[1])..., Tuple(bell_scenario[4] for _ in 1:seq_scenario[2])..., Tuple(bell_scenario[1] for _ in 1:seq_scenario[1])...,  Tuple(bell_scenario[2] for _ in 1:seq_scenario[2])...)) "Scenario does not match dimensions of full joint distribution"
        @show sum(condit_joint_dist, dims=Tuple(1:seq_scenario[1]+seq_scenario[2]))
        @assert all(abs.(sum(condit_joint_dist, dims=Tuple(1:seq_scenario[1]+seq_scenario[2])) .- 1.0) .< global_eps_tol) #Normalization constraint on P(a,b|x,y) for all (x,y) pairs

        #P[a1, a2, ..., a_s, b1, b2, ..., b_t, x1, x2, ..., xs, y1, y2, ..., yt]
        first_A_output_dim, first_A_input_dim = 1, (seq_scenario[1]+seq_scenario[2])+1
        first_B_output_dim, first_B_input_dim = seq_scenario[1]+1,(seq_scenario[1]+seq_scenario[2])+seq_scenario[1]+1

        all_A_marginals = sum(condit_joint_dist, dims=Tuple(first_B_output_dim:first_B_output_dim-1+seq_scenario[2])) #dims stay alive!
        all_B_marginals = sum(condit_joint_dist, dims=Tuple(first_A_output_dim:first_A_output_dim-1+seq_scenario[1]))
        
        #Check No-signaling B->A
        @assert all(abs.(all_A_marginals .- dynamic_slice(all_A_marginals, collect(first_B_input_dim:first_B_input_dim-1+seq_scenario[2]))) .< global_eps_tol) #Check that all A marginals are the same for all y input-dimensions.
        
        #Check Arrow-of-Time A
        for z in 1:seq_scenario[1]
            #Marginalize away all z+1 to s output-dimensions a
            present_and_past_A_marginals = sum(condit_joint_dist, dims=Tuple(z+1:seq_scenario[1]))
            
            #Check independence of all z+1 to s output-dimensions x
            @assert all(abs.(present_and_past_A_marginals .- dynamic_slice(present_and_past_A_marginals, collect(first_A_input_dim+(z+1):first_A_input_dim-1+seq_scenario[1]))) .< global_eps_tol)
        end

        #Check No-signaling A->B
        @assert all(abs.(all_B_marginals .- dynamic_slice(all_B_marginals, collect(first_A_input_dim:first_A_input_dim-1+seq_scenario[1]))) .< global_eps_tol) #Check that all B marginals are the same for all x input-dimensions.
        
        #Check Arrow-of-Time B
        for z in 1:seq_scenario[2]
            #Marginalize away all z+1 to s output-dimensions a
            present_and_past_B_marginals = sum(condit_joint_dist, dims=Tuple(first_B_output_dim+(z+1):first_B_output_dim-1+seq_scenario[2]))
          
            #Check independence of all z+1 to s output-dimensions x
            @assert all(abs.(present_and_past_B_marginals .- dynamic_slice(present_and_past_B_marginals, collect(first_B_input_dim+(z+1):first_B_input_dim-1+seq_scenario[2]))) .< global_eps_tol) 
        end


        new(bell_scenario, seq_scenario, condit_joint_dist)
    end
end

Base.:*(x::Real, y::TimeOrderedNSBox) = y=>x
Base.:*(x::TimeOrderedNSBox, y::Real) = x=>y

function Base.:+(x::Pair{TimeOrderedNSBox, <:Real}, y::Pair{TimeOrderedNSBox, <:Real})
    """Mixing of two TimeOrderedNSBoxes with asscoiated probabilities
    """
    (tonsbox_1, coeff_1) = x
    (tonsbox_2, coeff_2) = y

    #Check that both NSBoxes have the same scenario:
    @assert tonsbox_1.bell_scenario == tonsbox_2.bell_scenario "NSBoxes can only be mixed if they are in the same scenario"
    @assert abs(coeff_1 + coeff_2 - 1.0) < global_eps_tol

    mixed_joint_dist = coeff_1 * tonsbox_1.condit_joint_dist + coeff_2 * tonsbox_2.condit_joint_dist
    return TimeOrderedNSBox(bell_scenario=tonsbox_1.bell_scenario, seq_scenario=tonsbox_1.seq_scenario, condit_joint_dist=mixed_joint_dist)
end

In [24]:

function extremal_B_tonsbox_chsh(input_nsboxes::Pair{Symbol, <:Tuple}...)
    #nsboxes = expect all to be either :LD=>(α,γ,β,λ) box or :PR=>(μ,ν,σ) (only one :PR is allowed) -> User is trusted on this.
    #If all nsboxes are LD, then only the first nsbox is used to determine the behavior of the opposite party to "side".
    #Although LD nsboxes are bipartite correlations, the correlations of party != "side" will be ignored. (i.e. by default B's LD correlations are ignored)
    
    bell_scenario= (2,2,2,2)
    seq_scenario = (1, length(input_nsboxes))
    includes_pr = count([nb.first for nb in input_nsboxes] .== :PR)
    @assert includes_pr <= 1 "At most one PR box is allowed in TO NSBoxes"
    
    extremal_box = ones(Tuple(2 for _ in 1:(2*(seq_scenario[1]+seq_scenario[2])))...) #Might replace by a different init value.
    
    first_input_dim = (seq_scenario[1] + seq_scenario[2] + 1)

    for (i_b, box) in enumerate(input_nsboxes)
        if box.first == :PR
            @assert length(box.second) == 3 "PR box must have 3 parameters (μ,ν,σ)"
            box_full_joint = nsboxes.reconstructFullJoint(PRBoxesCHSH(;Dict(zip((:μ,:ν,:σ), box.second))...))
            reshape_size = Tuple(dim in (1, first_input_dim, 1+i_b, first_input_dim+i_b) ? size(extremal_box, dim) : 1 for dim in 1:ndims(extremal_box))
            extremal_box .*= reshape(box_full_joint, reshape_size)
        elseif box.first == :LD
            @assert length(box.second) == 4 "LD box must have 4 parameters (α,γ,β,λ)"
            box_full_joint = nsboxes.reconstructFullJoint(LocalDeterministicBoxesCHSH(;Dict(zip((:α,:γ,:β,:λ), box.second))...))
            
            if includes_pr == 0 && i_b == 1
                #Use the full nsbox
                reshape_size = Tuple(dim in (1, first_input_dim, 1+i_b, first_input_dim+i_b) ? size(extremal_box, dim) : 1 for dim in 1:ndims(extremal_box))
                extremal_box .*= reshape(box_full_joint, reshape_size)
            else
                #Only use the "side" part of the nsbox
                reshape_size = Tuple(dim in (1+i_b, first_input_dim+i_b) ? size(extremal_box, dim) : 1 for dim in 1:ndims(extremal_box))
                println(reshape_size)
                extremal_box .*= reshape(sum(box_full_joint, dims=1)[1,:,1,:], reshape_size) #A's and B's dimensions shouldn't depend on each other for LD boxes.
            end

        else
            error("Unknown box type $(box.first)")
        end
    end
    
    #TODO: Might need re-normalization after artificially sticking correlations together. But then normalization check to-box creation should indicate this need!
    
    return TimeOrderedNSBox(bell_scenario=bell_scenario, seq_scenario=seq_scenario, condit_joint_dist=extremal_box)
end

extremal_B_tonsbox_chsh (generic function with 1 method)

In [25]:
v1 = extremal_B_tonsbox_chsh( :PR=>(0,0,0),:LD=>(0,0,0,0)).condit_joint_dist

(1, 1, 2, 1, 1, 2)
sum(condit_joint_dist, dims = Tuple(1:seq_scenario[1] + seq_scenario[2])) = [1.0;;;; 1.0;;;;; 1.0;;;; 1.0;;;;;; 1.0;;;; 1.0;;;;; 1.0;;;; 1.0]


2×2×2×2×2×2 Array{Float64, 6}:
[:, :, 1, 1, 1, 1] =
 0.5  0.0
 0.0  0.5

[:, :, 2, 1, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 1, 1] =
 0.5  0.0
 0.0  0.5

[:, :, 2, 2, 1, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 1, 2, 1] =
 0.5  0.0
 0.0  0.5

[:, :, 2, 1, 2, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 2, 1] =
 0.0  0.5
 0.5  0.0

[:, :, 2, 2, 2, 1] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 1, 1, 2] =
 0.5  0.0
 0.0  0.5

[:, :, 2, 1, 1, 2] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 1, 2] =
 0.5  0.0
 0.0  0.5

[:, :, 2, 2, 1, 2] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 1, 2, 2] =
 0.5  0.0
 0.0  0.5

[:, :, 2, 1, 2, 2] =
 0.0  0.0
 0.0  0.0

[:, :, 1, 2, 2, 2] =
 0.0  0.5
 0.5  0.0

[:, :, 2, 2, 2, 2] =
 0.0  0.0
 0.0  0.0

In [26]:


function inefficient_extremal_B_tonsbox_chsh(input_nsboxes::Pair{Symbol, <:Tuple}...)

    bell_scenario= (2,2,2,2)
    seq_scenario = (1, length(input_nsboxes))

    base_boxes = [box.first == :PR ? nsboxes.reconstructFullJoint(PRBoxesCHSH(;Dict(zip((:μ,:ν,:σ), box.second))...)) : nsboxes.reconstructFullJoint(LocalDeterministicBoxesCHSH(;Dict(zip((:α,:γ,:β,:λ), box.second))...)) for box in input_nsboxes]
    extremal_box = ones(Tuple(2 for _ in 1:(2*(seq_scenario[1]+seq_scenario[2])))...) #Might replace by a different init value.
    
    first_input_dim = (seq_scenario[1] + seq_scenario[2] + 1)

    for a in 1:2
        for bs in Iterators.product((1:2 for _ in 1:seq_scenario[2])...)
            for x in 1:2
                for ys in Iterators.product((1:2 for _ in 1:seq_scenario[2])...)
                    extremal_box[a, bs..., x, ys...] = prod(input_nsboxes[i_b].first == :PR ? base_boxes[i_b][a, bs[i_b], x, ys[i_b]] : sum(base_boxes[i_b], dims=1)[1, bs[i_b], 1, ys[i_b]] for i_b in 1:length(input_nsboxes))
                end
            end
        end
    end
    
    return TimeOrderedNSBox(bell_scenario=bell_scenario, seq_scenario=seq_scenario, condit_joint_dist=extremal_box)
end

inefficient_extremal_B_tonsbox_chsh (generic function with 1 method)

In [27]:
v2 = inefficient_extremal_B_tonsbox_chsh(:PR=>(0,0,0), :LD=>(0,0,0,0)).condit_joint_dist
all(v1 .== v2)

sum(condit_joint_dist, dims = Tuple(1:seq_scenario[1] + seq_scenario[2])) = [1.0;;;; 1.0;;;;; 1.0;;;; 1.0;;;;;; 1.0;;;; 1.0;;;;; 1.0;;;; 1.0]


true

In [28]:
v_mix = 0.5*extremal_B_tonsbox_chsh( :PR=>(0,0,0),:LD=>(0,0,0,0)) + 0.5*extremal_B_tonsbox_chsh( :LD=>(0,0,0,0), :PR=>(0,0,0))

(1, 1, 2, 1, 1, 2)
sum(condit_joint_dist, dims = Tuple(1:seq_scenario[1] + seq_scenario[2])) = [1.0;;;; 1.0;;;;; 1.0;;;; 1.0;;;;;; 1.0;;;; 1.0;;;;; 1.0;;;; 1.0]
(1, 2, 1, 1, 2, 1)
sum(condit_joint_dist, dims = Tuple(1:seq_scenario[1] + seq_scenario[2])) = [1.0;;;; 1.0;;;;; 1.0;;;; 1.0;;;;;; 1.0;;;; 1.0;;;;; 1.0;;;; 1.0]
sum(condit_joint_dist, dims = Tuple(1:seq_scenario[1] + seq_scenario[2])) = [1.0;;;; 1.0;;;;; 1.0;;;; 1.0;;;;;; 1.0;;;; 1.0;;;;; 1.0;;;; 1.0]


TimeOrderedNSBox((2, 2, 2, 2), (1, 2), [0.5 0.0; 0.0 0.25;;; 0.0 0.0; 0.25 0.0;;;; 0.5 0.0; 0.0 0.25;;; 0.0 0.0; 0.25 0.0;;;;; 0.5 0.0; 0.0 0.25;;; 0.0 0.0; 0.25 0.0;;;; 0.25 0.25; 0.25 0.0;;; 0.0 0.0; 0.25 0.0;;;;;; 0.5 0.0; 0.0 0.25;;; 0.0 0.0; 0.25 0.0;;;; 0.25 0.0; 0.25 0.25;;; 0.25 0.0; 0.0 0.0;;;;; 0.5 0.0; 0.0 0.25;;; 0.0 0.0; 0.25 0.0;;;; 0.0 0.25; 0.5 0.0;;; 0.25 0.0; 0.0 0.0])

In [29]:
size(v_mix.condit_joint_dist)

(2, 2, 2, 2, 2, 2)

#### Protocol Optimization

In [30]:
#using ModelingToolkit, Optimization, OptimizationOptimJL

using JuMP, HiGHS

#@variable(Model(), p in Parameter(4.0))
#parameter_value.(p)
# set_parameter_value(p[2], 3.0)

In [31]:

# Bell (2,2,2,2) scenario
bell_scene = (2,2,2,2) #(n,n,2,2)
seq_scnee = (1, 2) #(1,t)
n, t = bell_scene[2], seq_scnee[2]
@assert n == 2 "Only n=2 is supported for now"
@assert t == 1 || t == 2 || t == 3 "Only t=1 or t=2 or t=3 is supported for now"

seq_RAC_model = Model(HiGHS.Optimizer)

@variable(seq_RAC_model, η in Parameter(0.5)) #Mixing parameter TONS-box

@variable(seq_RAC_model, Χ[0:1, 0:1], Bin) #Χ(α0, α1)

@variable(seq_RAC_model, E[0:1, 0:1, 0:1], Bin) #E(α0, α1, a)

D = []
### Note: For structure of D() it is important that all inputs and outputs (y and b) are available to all guessers, but guesses are made sequentially and a time-ordered structure needs to be ensured!
for z in 0:t-1
    if z == 0
        push!(D, eval(:(@variable(seq_RAC_model, $(Expr(:ref, :D0, :(0:1), [:(0:1) for _ in 0:t-1]..., [:(0:1) for _ in 0:t-1]...)), Bin)))) #D^(z)(E, b_0, ..., b_t-1, y_0, ..., y_t-1) with z ∈ [0,t-1]
    else
        push!(D, eval(:(@variable(seq_RAC_model, $(Expr(:ref, Symbol("D"*string(z)), :(0:1), [:(0:1) for _ in 0:t-1]..., [:(0:1) for _ in 0:t-1]..., [:(0:1) for _ in 0:z-1]...)), Bin)))) #D^(z)(E, b_0, ..., b_t-1, y_0, ..., y_t-1, g^0, ..., g^z-1) with z ∈ [0,t-1]
    end
end


_ = nothing #To avoid printing the output

In [33]:

function get_seq_RAC_MutInfo_Bound_CHSH(bell_scenario::NTuple{4, <:Int}, seq_scenario::NTuple{2, <:Int})

    #Bell scenario (n,n,2,2)
    #@assert bell_scenario[1] == bell_scenario[2]
    #@assert bell_scenario[3] == 2 && bell_scenario[4] == 2
    @assert bell_scenario == (2,2,2,2) #Only supported scenario for now

    #Sequence scenario (1,t)
    @assert seq_scenario[1] == 1

    plogp(p) = p == 0 ? 0 : p * log2(p)

    return (η, Χ, E, D) -> begin
        tons_box = η * extremal_B_tonsbox_chsh( :PR=>(0,0,0),:LD=>(0,0,0,0)) + (1-η) * extremal_B_tonsbox_chsh( :LD=>(0,0,0,0), :PR=>(0,0,0))

        ### Χ = Χ(\vec{α}) |-> [n]
        ### E = E(\vec{α},a) |-> [2]
        ### D = [D^0(E(\vec{α},a),b_0, b_1, y_0, y_1), D^1(E(\vec{α},a), b_0, b_1, y_0, y_1, g^(0)), ...] |-> [2, 2, ...]
        Χ_func(α) = Χ[...]

        
        cumul_mutinfo = 0.0
        
        z = 0
        P0_g = Array{Float64}(undef, 2, fill(2, n)..., n)  #P0_g = P(g^(0)|\vec{α} = \vec{v}, y_0 = i)
        for i in 0:n-1
            for v_vec in Iterators.product(fill(0:1, n)...)
                for c in 0:1
                    P0_g[c, v_vec... , i] = 1/2 + (e_c/2)*( sum( (D0_func(E_func(v_vec, a),b_0, i)==c) for a in 0:1 for b_0 in 0:1) )
                end
            end
        end
        

        cumul_mutinfo_z0 = n
        for i in 0:n-1
            TI = -sum( plogp((1/(2^n))*sum(P0_g[c,v_vec...,i] for v_vec in Iterators.product(fill(0:1, n)...) )) for c in 0:1)
            TII = -sum( plogp((1/(2^n))*P0_g[c,v_vec...,i] ) for c in 0:1 for v_vec in Iterators.product(fill(0:1, n)...))
            cumul_mutinfo_z0 = (1/(n^2)) * (TI - TII)
        end
        

        z=1
        P1_g #P1_g = P(g^(1), g^(0)|\vec{α} = \vec{v}, y_0 = j_0, y_1 = i)

        cumul_mutinfo_z1 = n
        for j_0 in 0:n-1
            for i in 0:n-1
                continue
            end
        end

        cumul_mutinfo
    end
end

seq_RAC_mutinfo_bound = get_seq_RAC_MutInfo_Bound(bell_scene, seq_scene)
@objective(seq_RAC_model, Max, seq_RAC_mutinfo_bound(η, Χ, E, D))

Base.Meta.ParseError: ParseError:
# Error @ s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\notebooks\SeqRACProtocolSearch.ipynb:8:13
        cumul_mutinfo = 0.0
        for ...
#           └─┘ ── invalid identifier

In [15]:

@parameters η

loss(Χ, η) = η * Χ(1,1) + (1-η) * Χ(1,0)


LoadError: LoadError: UndefVarError: `@parameters` not defined
in expression starting at s:\Sync\University\2023_MRP_2\MRP2_WorkDir\ABoxWorld\notebooks\SeqRACProtocolSearch.ipynb:2

In [16]:
Χ

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, 0:1
    Dimension 2, 0:1
And data, a 2×2 Matrix{VariableRef}:
 Χ[0,0]  Χ[0,1]
 Χ[1,0]  Χ[1,1]

In [20]:
@variable(seq_RAC_model, Χ[0:1, 0:1], Bin) #Χ(α0, α1)

@variable(seq_RAC_model, E[0:1, 0:1, 0:1], Bin) #E(α0, α1, a)

eval(:(@variable(seq_RAC_model, $(Expr(:ref, :D, 0:t-1, :(0:1), [:(0:1) for _ in 1:t]..., [:(0:1) for _ in 1:t]..., [:(0:1) for _ in 1:t]...)), Bin))) #D(z, E, b0, ..., bt-1, y0, ..., yt-1, g0, ..., gt-1) with z ∈ [0,t-1] s.t. D[z, ...] = D^(z)


#Constraints for (2,2,2,2) scenario and t=1, t=2
if t == 1
    #No constraints needed for t=1
elseif t == 2
    for z in 0:1
        for c_E in 0:1
            for y in Iterators.product(0:1, 0:1)
                #Optional constraints == independence of b_i for i != z 
                
                for g in Iterators.product(0:1, 0:1)
                    if z == 0
                        @constraint(seq_RAC_model, [b_z = 0:1], D[z, c_E, b_z, 0, y[1], y[2], g[1], g[2]] == D[z, c_E, b_z, 1, y[1], y[2], g[1], g[2]]) #Independence of b_i for all i != z  
                    elseif z == 1
                        @constraint(seq_RAC_model, [b_z = 0:1], D[z, c_E, 0, b_z, y[1], y[2], g[1], g[2]] == D[z, c_E, 1, b_z, y[1], y[2], g[1], g[2]]) #Independence of b_i for all i != z
                    end
                end

                # Mandatory constraints == independence of g_z (i.e. itself)
                for b in Iterators.product(0:1, 0:1)
                    if z == 0
                        @constraint(seq_RAC_model, [g_not_z = 0:1], D[z, c_E, b[1], b[2], y[1], y[2], 0, g_not_z] == D[z, c_E, b[1], b[2], y[1], y[2], 1, g_not_z]) #Independence of g_0 (i.e. itself)
                    elseif z == 1
                        @constraint(seq_RAC_model, [g_not_z = 0:1], D[z, c_E, b[1], b[2], y[1], y[2], g_not_z, 0] == D[z, c_E, b[1], b[2], y[1], y[2], g_not_z, 1]) #Independence of g_1 (i.e. itself)
                    end
                end
            end
        end
    end
elseif t == 3
    for z in 0:2
        for c_E in 0:1
            for y in Iterators.product(0:1, 0:1, 0:1)
                #Optional constraints == independence of b_i for i != z 
                
                for g in Iterators.product(0:1, 0:1, 0:1) 
                    if z == 0
                        for b_red in Iterators.product(0:1, 0:1) #b_1, b_2 (not b_0)
                            @constraint(seq_RAC_model, [b_z = 0:1], D[z, c_E, b_z, b_red[1], b_red[2], y[1], y[2], y[3], g[1], g[2], g[3]] == D[z, c_E, b_z, b_red[1], b_red[2], y[1], y[2], y[3], g[1], g[2], g[3]]) #Independence of b_i for all i != z
                        end
                    elseif z == 1
                        for b_red in Iterators.product(0:1, 0:1) #b_0, b_2 (not b_1)
                            @constraint(seq_RAC_model, [b_z = 0:1], D[z, c_E, b_red[1], b_z, b_red[2], y[1], y[2], y[3], g[1], g[2], g[3]] == D[z, c_E, b_red[1], b_z, b_red[2], y[1], y[2], y[3], g[1], g[2], g[3]]) #Independence of b_i for all i != z
                        end
                    elseif z == 2
                        for b_red in Iterators.product(0:1, 0:1) #b_0, b_1 (not b_2)
                            @constraint(seq_RAC_model, [b_z = 0:1], D[z, c_E, b_red[1], b_red[2], b_z, y[1], y[2], y[3], g[1], g[2], g[3]] == D[z, c_E, b_red[1], b_red[2], b_z, y[1], y[2], y[3], g[1], g[2], g[3]]) #Independence of b_i for all i != z
                        end
                    end
                end
                
                for b in Iterators.product(0:1, 0:1, 0:1)
                    if z == 0
                        for g_red in Iterators.product(0:1, 0:1) #g_0 (not g_1, g_2)
                            @constraint(seq_RAC_model, D[z, c_E, b[1], b[2], b[3], y[1], y[2], y[3], 0, g_red[1], g_red[2]] == D[z, c_E, b[1], b[2], b[3], y[1], y[2], y[3], 1, g_red[1], g_red[2]]) #Independence of g_0 (i.e. itself)
                        end
                    elseif z == 1
                        for g_red in Iterators.product(0:1, 0:1) #g_1 (not g_0, g_2)
                            @constraint(seq_RAC_model, D[z, c_E, b[1], b[2], b[3], y[1], y[2], y[3], g_red[1], 0, g_red[2]] == D[z, c_E, b[1], b[2], b[3], y[1], y[2], y[3], g_red[1], 1, g_red[2]]) #Independence of g_1 (i.e. itself)
                        end
                    elseif z == 2
                        for g_red in Iterators.product(0:1, 0:1) #g_2 (not g_0, g_1)
                            @constraint(seq_RAC_model, D[z, c_E, b[1], b[2], b[3], y[1], y[2], y[3], g_red[1], g_red[2], 0] == D[z, c_E, b[1], b[2], b[3], y[1], y[2], y[3], g_red[1], g_red[2], 1]) #Independence of g_2 (i.e. itself)
                        end
                    end
                end
            end
        end 
    end
end


BoundsError: BoundsError: attempt to access Tuple{} at index [1]